<a href="https://colab.research.google.com/github/Zooshi/testt/blob/master/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
up = files.upload()

Saving house-prices-advanced-regression-techniques.zip to house-prices-advanced-regression-techniques.zip


In [2]:
!ls

house-prices-advanced-regression-techniques.zip  sample_data


In [3]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df.shape

(1460, 81)

In [16]:
df.loc[:,df.isnull().sum().values>0].columns

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [28]:
list(df.columns[df.isnull().sum().values>0])

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [19]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,OneHotEncoder
from sklearn.base import BaseEstimator,TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error, auc

In [21]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [29]:
trainsmall = train.drop(list(df.columns[df.isnull().sum().values>0]), axis=1)

In [32]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [33]:
x_train,x_val, y_train,y_val = train_test_split(trainsmall,y, test_size=0.2,random_state=11)

In [35]:
x_train_small = x_train.select_dtypes(include=['number'])
x_train_small.shape

(1168, 34)

In [36]:
model=GradientBoostingRegressor()
model.fit(x_train_small,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [37]:
model.score(x_train_small,y_train)

0.9625460068961941

In [45]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
  def __init__(self, models):
    self.models = models

  def fit(self,X,y):
    self.models_ = [clone(x) for x in self.models]
    for model in self.models_:
      model.fit(X,y)
    return self

  def predict(self,X):
    predictions = np.column_stack([
                                   model.predict(X) for model in self.models_
    ])
    return np.mean(predictions, axis=1)

In [46]:
Enet = ElasticNet()
GBoost = GradientBoostingRegressor()
lasso = Lasso()

first = AveragingModels(models=(Enet,GBoost,lasso))

In [47]:
first.fit(x_train_small,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 726060221303.757, tolerance: 703727950.1292837
  positive)


AveragingModels(models=(ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                   l1_ratio=0.5, max_iter=1000, normalize=False,
                                   positive=False, precompute=False,
                                   random_state=None, selection='cyclic',
                                   tol=0.0001, warm_start=False),
                        GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='ls', max_depth=3,
                                                  max_feat...
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                          

In [48]:
first.predict(x_train_small)

array([ 84284.97837163, 225604.82606831, 204668.98616712, ...,
       242497.19544077, 285964.05057706, 105491.84357878])

In [73]:
folds = 5
def rmse_cv(model):
  kf = KFold(folds,shuffle=True, random_state=11).get_n_splits(x_train_small)
  rmse = np.sqrt(-cross_val_score(model,x_train_small,y_train, scoring = "neg_mean_squared_error", cv=kf))
  return rmse

In [66]:
kf = KFold(folds,shuffle=True, random_state=11)
kf

KFold(n_splits=5, random_state=11, shuffle=True)

In [61]:

score = rmse_cv(first)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 464143233934.8072, tolerance: 536350914.37857133
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 600638762427.8213, tolerance: 613578338.4302741
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 594565478107.539, tolerance: 564386947.6258879
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 582906585683

In [62]:
score.mean()

32120.61635891479

In [69]:
class Stacking(BaseEstimator,RegressorMixin,TransformerMixin):
  def __init__(self, base_models, meta_model, n_folds=5):
    self.base_models = base_models
    self.meta_model = meta_model
    self.n_folds = n_folds

  def fit(self, X,y):
    self.base_models_ = [list() for x in self.base_models]
    self.meta_model_ = clone(self.meta_model)
    kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=11)

    out_of_fold = np.zeros((X.shape[0], len(self.base_models)))
    for i,model in enumerate(self.base_models):
      for train_index, holdout_index in kfold.split(X,y):
        instance = clone(model)
        instance.fit(X[train_index].values,y[train_index].values)
        pred = instance.predict(X[holdout_index].values)
        out_of_fold[hold_out_index,i] = pred


    self.meta_model_.fit(out_of_fold,y)
    return self


  def predict(self,X):
    meta_features = np.column_stack([np.column_stack([model.predict(X) for model in base_models]).mean(axis=1) for base_models in self.base_models_])
    return self.meta_model_.predict(meta_features)

In [74]:
stack = Stacking(base_models=(Enet,lasso,GBoost), meta_model=lasso)
score = rmse_cv(stack)
print(score)

[nan nan nan nan nan]


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: "None of [Int64Index([  0,   1,   3,   5,   6,   7,   8,   9,  10,  11,\n            ...\n            921, 922, 923, 924, 926, 928, 929, 930, 932, 933],\n           dtype='int64', length=747)] are in the [columns]"

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: "None of [Int64Index([  0,   1,   3,   5,   6,   7,   8,   9,  10,  11,\n            ...\n            921, 923, 924, 927, 928, 929, 930, 931, 933, 934],\n           dtype='int64', length=748)] are in the [columns]"

  FitFailedWarning)


In [71]:
stack.fit(x_train_small.values,y_train.values)

AttributeError: ignored

In [89]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, cross_val_predict
from collections import defaultdict

In [87]:
X, y = make_classification(n_samples=10000, 
                           n_features=20, 
                           n_informative=15, 
                           n_redundant=5,
                           n_clusters_per_class=5,
                           class_sep=0.7,
                           flip_y=0.03,
                           n_classes=2)

In [88]:
models_dict = {'random_forest':     RandomForestClassifier(n_estimators=50),
               'svm': SVC(),
               'knn': KNeighborsClassifier(n_neighbors=11)}

In [97]:
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_predict(model, X, y, cv=cv.get_n_splits(), verbose=1, n_jobs=3)
    return scores
model_scores = {}
for name, model in models_dict.items():
    print('Evaluating {}'.format(name))
    scores = evaluate_model(model, X, y)
    model_scores[name] = scores

Evaluating random_forest


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:   15.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Evaluating svm


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:   27.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Evaluating knn


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    2.4s finished


In [ ]:
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=3, error_score='raise')
    return scores
model_scores = defaultdict()
for name, model in models_dict.items():
    print('Evaluating {}'.format(name))
    scores = evaluate_model(model, X, y)
    model_scores[name] = scores

In [96]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
cv.get_n_splits()

10

In [98]:
model_scores

{'knn': array([1, 1, 1, ..., 0, 0, 0]),
 'random_forest': array([1, 1, 1, ..., 1, 0, 0]),
 'svm': array([1, 1, 1, ..., 1, 0, 0])}

In [100]:
len(model_scores['knn'])

10000

In [101]:
df = pd.DataFrame(model_scores)
df.head()

,random_forest,svm,knn
0,1,1,1
1,1,1,1
2,1,1,1
3,1,0,0
4,0,0,0


In [104]:
def evaluate_model3(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=3, error_score='raise')
    return scores
model_scores = defaultdict()
for name, model in models_dict.items():
    print('Evaluating {}'.format(name))
    scores = evaluate_model3(model, X, y)
    model_scores[name] = scores

Evaluating random_forest


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:   12.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Evaluating svm


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:   24.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Evaluating knn


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    4.3s finished


In [105]:
model_scores

defaultdict(None,
            {'knn': array([0.8345, 0.8315, 0.853 , 0.834 , 0.836 , 0.844 , 0.816 , 0.8345,
                    0.8285, 0.858 ]),
             'random_forest': array([0.806 , 0.8095, 0.8245, 0.8215, 0.8155, 0.8275, 0.808 , 0.801 ,
                    0.816 , 0.833 ]),
             'svm': array([0.8285, 0.845 , 0.853 , 0.845 , 0.8505, 0.8585, 0.8295, 0.836 ,
                    0.852 , 0.853 ])})

In [107]:
from sklearn.linear_model import LogisticRegression

In [109]:
for model, score in model_scores.items():
  print(model , np.mean(score))

random_forest 0.8162499999999999
svm 0.8451000000000001
knn 0.837


In [110]:
df['true'] = y

In [114]:
lr = LogisticRegression()
lr.fit(df.iloc[:,:3],df['true'].values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [116]:
resnew = cross_val_score(lr,df.iloc[:,:3], df['true'].values)
resnew.mean()

0.8553000000000001

In [ ]:
preds = 

In [112]:
df.iloc[:,:3]


,random_forest,svm,knn
0,1,1,1
1,1,1,1
2,1,1,1
3,1,0,0
4,0,0,0
...,...,...,...
9995,1,1,1
9996,0,0,0
9997,1,1,0
9998,0,0,0


In [117]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV
base_models = [('random_forest', RandomForestClassifier(n_estimators=50)),
               ('svm', SVC()),
               ('knn', KNeighborsClassifier(n_neighbors=11))]
meta_model = LogisticRegressionCV()
stacking_model = StackingClassifier(estimators=base_models, 
                                    final_estimator=meta_model, 
                                    passthrough=True, 
                                    cv=5,
                                    verbose=2)

In [119]:
stacking_scores = evaluate_model(stacking_model, X, y)
stacking_scores

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  3.8min finished


array([1, 1, 1, ..., 1, 0, 0])

In [120]:
from sklearn.metrics import accuracy_score,classification_report




In [122]:
print(classification_report(df['true'],stacking_scores))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86      4999
           1       0.86      0.86      0.86      5001

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

